# This code only works on google collab

### Script to load the indexer already created + use the searcher

In [1]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2835, done.
remote: Counting objects: 100% (1179/1179), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 2835 (delta 957), reused 829 (delta 828), pack-reused 1656 (from 2)
Receiving objects: 100% (2835/2835), 2.07 MiB | 8.18 MiB/s, done.
Resolving deltas: 100% (1788/1788), done.


In [2]:
try:
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/
    !pip install faiss-gpu torch
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, use conda or install with pip, but it will be lower and less stable")
    assert False

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  DEPRECATION: Legacy editable install of colbert-ai==0.2.20 from file:///content/ColBERT (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop fo

In [3]:
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


## Unzip the archive containing the embeddings

In [4]:
import zipfile
import pandas as pd


with zipfile.ZipFile('experiments.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [5]:
import pickle

collection_file = 'experiments/movie_plots/indexes/wikipedia_movie_plots/movie_plots_collection.pkl'

with open(collection_file, 'rb') as f:
    collection = pickle.load(f)
    print("Collection chargée.")

Collection chargée.


### Searcher

In [10]:
from colbert import Searcher

nb_result = 3

index_name = "wikipedia_movie_plots"
experiments_name = "movie_plots"

# Load the dense index and configure the searcher
with Run().context(RunConfig(experiment=experiments_name)):
    searcher = Searcher(index=index_name, collection=collection)

query = "A movie about space exploration and human survival"
print(f"Requete: {query}")

results = searcher.search(query, k=nb_result)
print(f"Résultats: {results}")

id = results[0]
rank = results[1]
score = results[2]

[Jan 11, 16:24:33] #> Loading codec...
[Jan 11, 16:24:33] #> Loading IVF...
[Jan 11, 16:24:33] #> Loading doclens...


/content/ColBERT/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/content/ColBERT/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loa

[Jan 11, 16:24:34] #> Loading codes and residuals...



  0%|          | 0/1 [00:00<?, ?it/s]/content/ColBERT/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cod

Requete: A movie about space exploration and human survival

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: A movie about space exploration and human survival, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 1037, 3185, 2055, 2686, 8993, 1998, 2529, 7691,  102,  103,
         103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

Résultats: ([2080, 4457, 12749], [1, 2, 3], [19.37470245361328, 18.967586517333984, 18.755653381347656])


### Result

In [11]:
for i in range(nb_result):
    print(f"[Rank {rank[i]}] (Score: {score[i]:.2f})")
    print(collection[id[i]])
    print("\n\n")

[Rank 1] (Score: 19.37)
Title: Spaceflight IC-1: An Adventure in Space Plot: The opening narrative is given by a man in a high ranking military uniform. He tells us the film is set 50 years into the future, when an overcrowded Earth will begin looking for other Earths. Flights will be made taking 25 to 50 years. This film is the story of one such flight IC 1 (Interstellar Colony ship One) It is a Noahs Ark of space, 8 couples with their children will travel to Earth 2 well beyond our Solar System
We are then introduced to the couples who make up the crew. First is the Captain, Mead Ralston, (Bill Williams) his wife and educator Jan Ralston (Norma West) The medical team are doctors Steven Thomas (John Cairney) and his wife Helen Thomas (Linda Marlowe) Aside from the health of the crew, the doctors are involved in an experiment in suspended animation. Two men and two women to be revived when they reach Earth 2The next couple is engineer John Saunders (Jeremy Longhurst) and his wife biolo